In [1]:
import pymongo
from pymongo import MongoClient
from flask import(
    Flask,
    render_template,
    jsonify,
    request)


app = Flask(__name__) 

In [2]:
client = MongoClient('mongodb://localhost:27017')
# Get the sampleDB database
db = client.nba
nba_bio_collection = db.nba_bio 
nba_collg_collection = db.nba_collg

In [3]:
@app.route("/")
@app.route("/index")
def home():
    return(render_template("index.html"))

@app.route("/charts")
def charts():
    return(render_template("charts.html"))

@app.route("/form")
def forms():
    return(render_template("form.html"))

@app.route("/data")
def data():
    return(render_template("data.html"))

@app.route("/nba-bio") 
def pymongo_nba_bio_display():
    nba_bio_result=[]
    cursor = nba_bio_collection.find({})
    for document in cursor:
        document.pop("_id")
        nba_bio_result.append(document)
    return jsonify(nba_bio_result)

@app.route("/nba-collg") 
def pymongo_nba_collg_display():
    nba_collg_result=[]
    cursor = nba_collg_collection.find({})
    for document in cursor:
        document.pop("_id")
        nba_collg_result.append(document)    
    return jsonify(nba_collg_result)

@app.route("/treevariable")
def treevariable():
    # Get the nba_collg collection stored in a list
    Collg=[]
    cursor = nba_collg_collection.find({})
    for document in cursor:
        document.pop("_id")
        Collg.append(document)

    # Get the unique list of colleges 
    college_list=[]
    for i in range(0,len(Collg)):
        # Only If the document has keys or if the College is not NA
        if (("College" in Collg[i].keys()) and (Collg[i]["College"] not in college_list) and (Collg[i]["College"]!="NA")):
            college_list.append(Collg[i]["College"])

    # Start creating the variable for tree format
    collg_container=[]
    for i in range(0,len(college_list)):
        # Find the players from each college
        playersbycolg=[]
        for j in range(0,5):
            if(("College" in Collg[j].keys()) and (Collg[j]["College"]==college_list[i])):
                if(Collg[j]["picurl"]=="NA"):
                    playerstats = Collg[j]["collg-stats"]
                    player_info={"text":{
                                    "name":Collg[j]["FirstName"]+" "+Collg[j]["LastName"]
                                    },
                            "collapsed":"false",
                            "children":{
                                    "innerHTML":"<p><strong>PTS: </strong>"+playerstats["pts_per_g"]+"</p>"+
                                        "<p><strong>AST: </strong>"+playerstats["ast_per_g"]+"</p>"+
                                        "<p><strong>TRB: </strong>"+playerstats["trb_per_g"]+"</p>"+
                                        "<p><strong>MPG: </strong>"+playerstats["mp_per_g"]+"</p>"+
                                        "<p><strong>FG%: </strong>"+playerstats["fg_pct"]+"</p>"+
                                        "<p><strong>3P%: </strong>"+playerstats["fg3_pct"]+"</p>"+
                                        "<p><strong>FT%: </strong>"+playerstats["ft_pct"]+"</p>",
                                    "collapsed":"true",
                                    "children":{"text":"Statistics"}
                            }}
                else:
                    playerstats = Collg[j]["collg-stats"]
                    player_info={"image":Collg[j]["picurl"],
                                "text":{
                                    "name":Collg[j]["FirstName"]+" "+Collg[j]["LastName"]
                                        },
                                "collapsed":"true",
                                "children":{
                                     "innerHTML": "<p><strong>PTS: </strong>"+playerstats["pts_per_g"]+"</p>"+
                                        "<p><strong>AST: </strong>"+playerstats["ast_per_g"]+"</p>"+
                                        "<p><strong>TRB: </strong>"+playerstats["trb_per_g"]+"</p>"+
                                        "<p><strong>MPG: </strong>"+playerstats["mp_per_g"]+"</p>"+
                                        "<p><strong>FG%: </strong>"+playerstats["fg_pct"]+"</p>"+
                                        "<p><strong>3P%: </strong>"+playerstats["fg3_pct"]+"</p>"+
                                        "<p><strong>FT%: </strong>"+playerstats["ft_pct"]+"</p>",
                                    "collapsed":"true",
                                    "children":{"text":"Statistics"}
                                }}
                # Append the playerinfo to each colg (playersbycolg)
                playersbycolg.append(player_info)
        # Create a parent element as colg and child element as its players
        colg_child = {"text":{"name":college_list[i]},
                      "collapsed":"true",
                      "children":playersbycolg}
        # Append the colg child element into a list 
        collg_container.append(colg_child)
    # Create a parent element as NBA and child element as the list of child colleges
    node={"image":"http://icons.iconarchive.com/icons/chrisbanks2/cold-fusion-hd/128/nba-2-icon.png",
          "collapsed":"true",
            "children":collg_container}    
    return jsonify(node)

In [ ]:
if __name__ == "__main__":
    app.run(debug = False) 

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Mar/2018 18:37:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Mar/2018 18:37:19] "GET /static/vendor/bootstrap/css/bootstrap.min.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Mar/2018 18:37:19] "GET /vendor/font-awesome/css/font-awesome.min.css HTTP/1.1" 404 -
127.0.0.1 - - [22/Mar/2018 18:37:19] "GET /vendor/datatables/dataTables.bootstrap4.css HTTP/1.1" 404 -
127.0.0.1 - - [22/Mar/2018 18:37:19] "GET /static/css/sb-admin.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Mar/2018 18:37:19] "GET /static/vendor/jquery/jquery.min.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Mar/2018 18:37:19] "GET /static/vendor/bootstrap/js/bootstrap.bundle.min.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Mar/2018 18:37:19] "GET /static/vendor/jquery-easing/jquery.easing.min.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Mar/2018 18:37:19] "GET /static/js/sb-admin.min.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Mar/2018 18:37:19] "GET /static/js/app.js HTTP/1.1" 200 -
127.0.0.1 - 

In [ ]:
# def treevariable():
#     # Get the nba_collg collection stored in a list
#     Collg=[]
#     cursor = nba_collg_collection.find({})
#     for document in cursor:
#         document.pop("_id")
#         Collg.append(document)

#     # Get the unique list of colleges 
#     college_list=[]
#     for i in range(0,len(Collg)):
#         # Only If the document has keys or if the College is not NA
#         if (("College" in Collg[i].keys()) and (Collg[i]["College"] not in college_list) and (Collg[i]["College"]!="NA")):
#             college_list.append(Collg[i]["College"])

#     # Start creating the variable for tree format
#     collg_container=[]
#     for i in range(0,len(college_list)):
#         # Find the players from each college
#         playersbycolg=[]
#         for j in range(0,5):
#             if(("College" in Collg[j].keys()) and (Collg[j]["College"]==college_list[i])):
#                 if(Collg[j]["picurl"]=="NA"):
#                     playerstats = Collg[j]["collg-stats"]
#                     player_info={"text":{
#                                     "name":Collg[j]["FirstName"]+" "+Collg[j]["LastName"]
#                                     },
#                             "collapsed":"true",
#                             "children":{
#                                     "innerHTML":"<p><strong>PTS: </strong>"+playerstats["pts_per_g"]+"</p>"+
#                                         "<p><strong>AST: </strong>"+playerstats["ast_per_g"]+"</p>"+
#                                         "<p><strong>TRB: </strong>"+playerstats["trb_per_g"]+"</p>"+
#                                         "<p><strong>MPG: </strong>"+playerstats["mp_per_g"]+"</p>"+
#                                         "<p><strong>FG%: </strong>"+playerstats["fg_pct"]+"</p>"+
#                                         "<p><strong>3P%: </strong>"+playerstats["fg3_pct"]+"</p>"+
#                                         "<p><strong>FT%: </strong>"+playerstats["ft_pct"]+"</p>"
#                             }}
#                 else:
#                     playerstats = Collg[j]["collg-stats"]
#                     player_info={"image":Collg[j]["picurl"],
#                                 "text":{
#                                     "name":Collg[j]["FirstName"]+" "+Collg[j]["LastName"]
#                                         },
#                                 "collapsed":"true",
#                                 "children":{
#                                      "innerHTML": "<p><strong>PTS: </strong>"+playerstats["pts_per_g"]+"</p>"+
#                                         "<p><strong>AST: </strong>"+playerstats["ast_per_g"]+"</p>"+
#                                         "<p><strong>TRB: </strong>"+playerstats["trb_per_g"]+"</p>"+
#                                         "<p><strong>MPG: </strong>"+playerstats["mp_per_g"]+"</p>"+
#                                         "<p><strong>FG%: </strong>"+playerstats["fg_pct"]+"</p>"+
#                                         "<p><strong>3P%: </strong>"+playerstats["fg3_pct"]+"</p>"+
#                                         "<p><strong>FT%: </strong>"+playerstats["ft_pct"]+"</p>"
#                                 }}
#                 # Append the playerinfo to each colg (playersbycolg)
#                 playersbycolg.append(player_info)
#         # Create a parent element as colg and child element as its players
#         colg_child = {"text":{"name":college_list[i]},
#                       "collapsed":"true",
#                       "children":playersbycolg}
#         # Append the colg child element into a list 
#         collg_container.append(colg_child)
#     # Create a parent element as NBA and child element as the list of child colleges
#     node={"image":"http://icons.iconarchive.com/icons/chrisbanks2/cold-fusion-hd/128/nba-2-icon.png",
#           "collapsed":"true",
#             "children":collg_container}    
    
#     #print(node)
#     # Convert the response into a json response
#     #nodejson=json.dumps(node,sort_keys=False,indent=4)
#     # Return the json response for tree variable


In [ ]:
# treevariable()